In [1]:
import numpy as np
import scipy as sp
import astropy as ap
from scipy import ndimage

In [2]:
#Need to learn how to rotate array

matr_1= [[0, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 0]]

matr_2= [[0, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 0]]

print("Rotation by 45 degrees")
print(ndimage.rotate(matr_1, 45, reshape=False))

print("Rotation by 55 degrees")
print(ndimage.rotate(matr_2, 55, reshape=False))



Rotation by 45 degrees
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 0 0 1 1]
 [1 1 0 0 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Rotation by 55 degrees
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 1 0 1 1]
 [0 1 1 0 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [3]:
rand_matr = np.random.rand(6,6)
print("Original Matrix")
print("Original Dimension = " + str(np.shape(rand_matr)))
print(rand_matr)
print("Rotated Matrix")
rot_matr = ndimage.rotate(rand_matr, 45, reshape=False)
print("Rotated Dimension = "+ str(np.shape(rot_matr)))
print(rot_matr)


Original Matrix
Original Dimension = (6, 6)
[[0.55911785 0.38565567 0.33915408 0.96240449 0.85172541 0.49169546]
 [0.69770285 0.93600307 0.36478286 0.41714304 0.16077046 0.5560917 ]
 [0.96547317 0.98052876 0.70079598 0.26371723 0.58756878 0.23655975]
 [0.59312673 0.87732727 0.33412575 0.5581224  0.98929264 0.27892787]
 [0.95172086 0.56380419 0.11357682 0.43773716 0.69484024 0.14831199]
 [0.46228076 0.45841274 0.49834428 0.6066905  0.73131712 0.15581361]]
Rotated Matrix
Rotated Dimension = (6, 6)
[[0.         0.         0.69066905 0.4248233  0.         0.        ]
 [0.         0.57648438 0.32159635 0.42339541 0.42351636 0.        ]
 [0.49735088 0.49831    0.38749956 0.54406595 0.97996933 0.30470976]
 [0.82170606 0.99563083 0.6894091  0.26272625 0.55685812 0.73040597]
 [0.         0.81118519 0.79627203 0.14328671 0.43862863 0.        ]
 [0.         0.         0.89029665 0.45569119 0.         0.        ]]


# Need to Consider Galaxy Selection Criteria
## LRG Criteria from Literature
1709.05024

LRG Paper Selects $M_\star>10^{11.3} M_\odot$ and reject satellite LRGs by discounting them if there is a more massive LRG within $1 h^-1 $Mpc and within radial velocity dist $|c \Delta z | < 1000 $km/s

LRG Pairs are chosen by finding all neighboring pairs of LRG's within $6-10 h^-1$ Mpc tangential distance, and within a proper radial distance of $\pm 6 h^-1$ Mpc

262,864 LRG Pairs out to $z\approx 0.4$

## Galaxy Selection from Literature
1709.10378v2

Using CMASS Galaxy Catalogue from SDSS DR12, galaxies were selected so that they are separated by up to $>20 h^-1$ Mpc

Projected Separation of $6-14 h^-1$ Mpc and line of sight separation of $>5 h^-1$ Mpc

~1,000,000 Galaxy Pairs found with mean separation $10 h^-1$ Mpc, which puts it well beyond host DM halo

